# Hyperparameters Tuning

In [ ]:
import numpy as np
np.random.seed(1)
import pandas as pd

import scipy

import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import make_pipeline

from sklearn.model_selection import cross_val_score, learning_curve, validation_curve

import matplotlib.pyplot as plt

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Load dataset

In [ ]:
df = pd.read_csv(
    'https://archive.ics.uci.edu/ml/'
    'machine-learning-databases'
    '/breast-cancer-wisconsin/wdbc.data',
    header=None
)
df.head()
df.describe()

In [ ]:
from sklearn.preprocessing import LabelEncoder

X, y = df.iloc[:, 2:].values, df.iloc[:, 1].values
le = LabelEncoder()
y = le.fit_transform(y)
le.classes_
len(y)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=.2, stratify=y, random_state=1
)
len(y_train), len(y_test)

### Combing preprocessing and final model in a Pipeline

In [ ]:
pipe_lr = make_pipeline(
    StandardScaler(),
    PCA(n_components=2),
    LogisticRegression()
)
pipe_lr.fit(X_train, y_train)
y_pred = pipe_lr.predict(X_test)
test_acc = pipe_lr.score(X_test, y_test)
print(f'Test accuracy: {test_acc:.3f}')

### Cross-Validation:
Nel processo di tuning degli iperparametri del modello, è cruciale non usare sempre lo stesso dataset di test, perchè questo porterebbe a trovare una soluzione ottima con alto bias.
La corretta procedura, invece, deve essere quella di usare un dataset di *validation* estratto dal dataset di training. Una possibilità sarebbe quella di usare la tecnica del *holdout*, in cui il dataset totale viene suddiviso in 3 sotto insiemi: [train, validation, test], in genere con percentuali del tipo [70%, 10%, 20%], rispettivamente. Tuttavia, questa tecnica dipende fortemente dal unico modo utilizzato per splittare il dataset.

Una pratica ancora più performante è la tecnica dello *Stratified Cross-Validation*. In tale tecnica, il dataset iniziale viene suddivo in train e test, successivamente (in fase di *evaluation*) il dataset di train viene suddiviso in K *folds*, per cui K-1 vengono usati come dataset di training e il K-esimo come validation. La procedura viene ripetuta un numero K di volte, in cui il K-esimo dataset di validation è sempre differente dal caso precedente. Lo score finale sarà ottenuto prendendo la media delle K iterazioni. Inoltre, il metodo assicura che, in ogni iterazione il numero delle classi rimane constante (*stratified*), in modo da trainare i K modelli, in situazioni simili, per quanto riguarda il numero di labels.

A fine della procedura, viene trainato un unico modello, usando le configurazioni degli iperparametri ottimi, ed utilizzando l'intero dataset di training (train + validation == 80% dei dati) e usando il dataset di test (mai utilizzato finora) per valutare l'errore di generalizzazione. 

In [ ]:
from sklearn.model_selection import StratifiedKFold

KFold = StratifiedKFold(n_splits=10).split(X_train, y_train)

scores = []
for k, (train, test) in enumerate(KFold):
    _ = pipe_lr.fit(X_train[train], y_train[train])
    scores.append(pipe_lr.score(X_train[test], y_train[test]))
    print(
        f'Fold {k+1:02} '
        f'Class distr. : {np.bincount(y_train[train])} '
        f'CV score: {scores[-1]:.3f}')


print(f'CV accuracy averaged: {np.mean(scores):.3f} +/- {np.std(scores):.3f}')

In [ ]:
# the same thing, but less verbose ...
scores = cross_val_score(
    estimator=pipe_lr,
    X=X_train, 
    y=y_train,
    cv=10, # 10-folds 
    n_jobs=-1 # exploits all available cpus
)
print(f'CV accuracy scores:\n{scores}\n')
print(f'CV accuracy: {np.mean(scores):.3f} +/- {np.std(scores):.3f}')

### Validation curve
La validation curve visualizza l'accuratezza, sul dataset di training e di validation, al variare dei valori di alcuni iperparametri del modello. Il suo plot aiuta ad individuare, in modo grafico, il range di valori entro i quali, gli iperparametri danno un accuratezza sul validation test ottima.

In [ ]:
def plot_validation_curve(estimator, X, y, param_range, param_name, cv=10):
    """Validation curve plot: shows accuracy values for different values of a select parameters.
    
    --Params
     - estimator: a scikit-learn estimator,
     - X, y: numpy ndarrays
     - param_range: List, the range values the paramete must explor
     - param_name: str, the name of the parameter
     -cv: int, the number of folds for the stratified cross validation
     """
    
    train_score, test_score = validation_curve(
        estimator=estimator,
        X=X,
        y=y,
        param_range=param_range,
        param_name=param_name,
        cv=cv
    )

    train_avg, train_std = np.mean(train_score, axis=1), np.std(train_score, axis=1)
    test_avg, test_std = np.mean(test_score, axis=1), np.std(test_score, axis=1)
    
    plt.plot(param_range, train_avg, color='blue', marker='o', markersize=5, label='Training Accuracy')
    plt.fill_between(param_range, train_avg + train_std, train_avg - train_std, alpha=.15, color='blue')
    plt.plot(param_range, test_avg, color='green', marker='s', markersize=5, linestyle='--', label='Validation Accuracy')
    plt.fill_between(param_range, test_avg + test_std, test_avg - test_std, alpha=.15, color='green')
    plt.legend()
    plt.grid()
    plt.xlabel(f"Parameter {param_name.split('__')[-1]}")
    plt.ylabel('Accuracy')
    plt.ylim([0.8, 1.0])
    plt.xscale('log')
    plt.show()

pipe_lr = make_pipeline(
    StandardScaler(),
    LogisticRegression(penalty='l2', max_iter=10000)
)
plot_validation_curve(pipe_lr, X_train, y_train, [0.001, 0.01, 0.1, 1.0, 10.0, 100.0], 'logisticregression__C')

### Grid Search
Un modello di ML ha due tipi di parametri: i parametri imparati durante la fase di training (weights, bias), e dei parametri che definiscono il modello e sono impostati in fase di progettazione (learning-rate, penalization values, il max depth number in un albero decisionale). Questi ultimi, vanno tunati al fine di rintracciare l'insieme di valori ottimi, in termini di accuratezza del modello, per questo si chiamano hyperparameters. 

La Grid Search è una tecnica *brute-force* esaustiva, che permette di trovare la combinazione di parametri ottimali, semplicemente provandone tutte le possibili combinazioni. 

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

pipe_svc = make_pipeline(
    StandardScaler(),
    SVC(random_state=1)
)

param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
param_grid = [
    {'svc__C': param_range,
    'svc__kernel': ['linear']},

    {'svc__C': param_range,
    'svc__gamma': param_range,
    'svc__kernel': ['rbf']}]

grid = GridSearchCV(
    estimator=pipe_svc,
    param_grid=param_grid,
    scoring='accuracy',
    cv=10,
    refit=True,
    n_jobs=-1
)

gs = grid.fit(X_train, y_train)
gs.best_score_
gs.best_params_

In [ ]:
clf = gs.best_estimator_
_ = clf.fit(X_train, y_train)
print(f'Test accuracy: {clf.score(X_test, y_test):.3f}')

### Random Search
Il metodo del grid search essendo un metodo *esaustivo* rintraccia la combinazione ottima degli iperparametri, tuttavia diventa ostico poterlo applicare quando ci sono più di 3 parametri da ottimizzare (too computational expensive), i parametri da ottimizzare possono assumere valori continui (molti valori saranno trascurati).
Inoltre, tutti i valori che si andranno ad esplorare sono fissati in modo deterministico, pertanto non esplorati in modo uniforme. 

La tecnica del random search, permette di migliorare le performance rispetto a questi punti. I valori da esplorare per gli iperarametri sono estratti da una distribuzione di probabilità (uniforme o altro tipo), e questo assicura di esplorare i valori in modo meno sistematico e quindi più efficiente *(un esplorazione degli iperparametri random è statisticamente più efficiente di una deterministica *Random Search for Hyper-Parameter Optimization by J. Bergstra, Y. Bengio, Journal of Machine Learning 
Research, pp. 281-305, 2012*)* 

In [ ]:
param_range = scipy.stats.loguniform(0.0001, 1000.0)
param_range.rvs(10)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = [
    {"svc__C": param_range,
    "svc__kernel": ["linear"]},
    {"svc__C": param_range,
    "svc__gamma": param_range,
    "svc__kernel": ["rbf"]}
]

rs = RandomizedSearchCV(
    estimator=pipe_svc,
    param_distributions=param_grid, # note the *ditributions part
    scoring='accuracy',
    refit=True,
    n_iter=20,
    cv=10,
    random_state=1,
    n_jobs=-1
)

_ = rs.fit(X_train, y_train)
rs.best_score_
rs.best_params_

notare come, l'uso del random search e quindi l'uso di una funzione di ditribuzione per il parametro c, ha permesso di rintracciare un valore di C, altrimenti escluso nella ricerca con il grid search, rintracciando una configuarazione ottimale più semplice (a livello computazionale più efficiente) ma che apporta grosso modo lo stesso valore di accuratezza, in fase di test, come si vede sotto.

In [ ]:
clf = rs.best_estimator_
_ = clf.fit(X_train, y_train)
print(f'Test accuracy: {clf.score(X_test, y_test):.3f}')

### Halving random search

Stesso concetto del random search, ma, usa le risorse (il dataset) in modo incrementale: inizia con tutti i candidati utilizzando metà del dataset, in successive iterazioni, seleziona solo i migliori candidati e incrementa il dataset.

In [ ]:
# è una features sperimentale: bisogna importare questa macro, perchè l'API potrebbe cambiare senza deprecation cycle
from sklearn.experimental import enable_halving_search_cv
# adesso importa il metodo
from sklearn.model_selection import HalvingRandomSearchCV

hs = HalvingRandomSearchCV(
    estimator=pipe_svc,
    param_distributions=param_grid,
    n_candidates='exhaust', # scegli un numero di candidati tale da sfruttare tutto il dataset
    resource='n_samples',
    factor=1.5, # quanti candidati saranno scartati ad ogni iterazione (100%/1.5=66% rimarranno)
    random_state=1,
    n_jobs=-1
    )

_ = hs.fit(X_train, y_train)
hs.best_score_
hs.best_params_


In [ ]:
clf = hs.best_estimator_
_ = clf.fit(X_train, y_train)
print(f'Test accuracy: {clf.score(X_test, y_test):.3f}')